# Create realisations from MAGICC's probabilistic output and MESMER

This uses data from the [RCMIP Phase 2 paper (in press)](https://doi.org/10.1002/essoar.10504793.2), archived [here](http://doi.org/10.5281/zenodo.4624566).

In [1]:
import joblib
import os.path

import mesmer.create_emulations
import scmdata
import scmdata.database
import xarray as xr
from openscm_units import unit_registry

<IPython.core.display.Javascript object>

/data/ubuntu-znicholls/miniconda3/envs/mesmer-openscmrunner/lib/python3.9/site-packages/scmdata/database.py:19: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  import tqdm.autonotebook as tqdman


In [ ]:
assert scmdata.__version__.startswith("0.9"),scmdata.__version__

## Configuration

In [2]:
SCENARIOS_TO_RUN = ["ssp*"]
N_REALISATIONS_PER_SCENARIO = 2  # careful making this big, there's 600 MAGICC probabilistic ensemble members
YEARS_TO_EMULATE = range(1850, 2150 + 1)
OUT_FILE = "probabilistic_example_emulation.nc"

## Load data

### MAGICC output

In [3]:
print("rcmip-phase-2-archive/rcmip-phase-2/data/processed/submission-database/MAGICCv7.5.1".replace("/", '",\n"'))

rcmip-phase-2-archive",
"rcmip-phase-2",
"data",
"processed",
"submission-database",
"MAGICCv7.5.1


In [4]:
!ls rcmip-phase-2-archive/rcmip-phase-2/data/processed/submission-database/MAGICCv7.5.1/Surface-Air-Temperature-Change/World/ssp126

MAGICCv7.5.1_Surface-Air-Temperature-Change_World_ssp126.nc


In [5]:
magicc_output_dir = os.path.join(
    ".",
    "rcmip-phase-2-archive",
    "rcmip-phase-2",
    "data",
    "processed",
    "submission-database",
)

In [6]:
db = scmdata.database.ScmDatabase(magicc_output_dir)
sorted(db.available_data()["variable"].unique())

['Atmospheric-Concentrations-CO2',
 'Atmospheric-Lifetime-CH4',
 'Atmospheric-Lifetime-N2O',
 'Carbon-Pool-Atmosphere',
 'Cumulative-Emissions-CO2',
 'Effective-Radiative-Forcing',
 'Effective-Radiative-Forcing-Anthropogenic-Aerosols',
 'Effective-Radiative-Forcing-Anthropogenic-Aerosols-Aerosols-cloud-Interactions',
 'Effective-Radiative-Forcing-Anthropogenic-Aerosols-Aerosols-radiation-Interactions',
 'Effective-Radiative-Forcing-Anthropogenic-CH4',
 'Effective-Radiative-Forcing-Anthropogenic-CO2',
 'Effective-Radiative-Forcing-Anthropogenic-F-Gases',
 'Effective-Radiative-Forcing-Anthropogenic-Montreal-Gases',
 'Effective-Radiative-Forcing-Anthropogenic-N2O',
 'Emissions-CO2',
 'Heat-Content-Ocean',
 'Heat-Uptake-Ocean',
 'Net-Land-to-Atmosphere-Flux-CO2',
 'Net-Ocean-to-Atmosphere-Flux-CO2',
 'Radiative-Forcing-Anthropogenic-Albedo-Change',
 'Surface-Air-Ocean-Blended-Temperature-Change',
 'Surface-Air-Temperature-Change']

In [7]:
magicc_gsat_output = scmdata.run_append([
    db.load(scenario=s, variable="Surface Air Temperature Change") for s in SCENARIOS_TO_RUN
])
display(magicc_gsat_output)

magicc_ohu_output = scmdata.run_append([
    db.load(scenario=s, variable="Heat Uptake|Ocean") for s in SCENARIOS_TO_RUN
])
display(magicc_ohu_output)

Loading files:   0%|          | 0/8 [00:00<?, ?it/s]

/data/ubuntu-znicholls/miniconda3/envs/mesmer-openscmrunner/lib/python3.9/site-packages/scmdata/netcdf.py:201: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dims["time"] = _read_time_variable(ds.variables["time"])


<scmdata.ScmRun (timeseries: 4800, timepoints: 456)>
Time:
	Start: 1850-01-01T00:00:00
	End: 2305-01-01T00:00:00
Meta:
	     climate_model  ensemble_member        model region scenario unit  \
	0     MAGICCv7.5.1                0  unspecified  World   ssp370    K   
	1     MAGICCv7.5.1                1  unspecified  World   ssp370    K   
	2     MAGICCv7.5.1                2  unspecified  World   ssp370    K   
	3     MAGICCv7.5.1                3  unspecified  World   ssp370    K   
	4     MAGICCv7.5.1                4  unspecified  World   ssp370    K   
	...            ...              ...          ...    ...      ...  ...   
	4795  MAGICCv7.5.1              595  unspecified  World   ssp245    K   
	4796  MAGICCv7.5.1              596  unspecified  World   ssp245    K   
	4797  MAGICCv7.5.1              597  unspecified  World   ssp245    K   
	4798  MAGICCv7.5.1              598  unspecified  World   ssp245    K   
	4799  MAGICCv7.5.1              599  unspecified  World   ssp245  

Loading files:   0%|          | 0/8 [00:00<?, ?it/s]

/data/ubuntu-znicholls/miniconda3/envs/mesmer-openscmrunner/lib/python3.9/site-packages/scmdata/netcdf.py:201: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dims["time"] = _read_time_variable(ds.variables["time"])


<scmdata.ScmRun (timeseries: 4800, timepoints: 455)>
Time:
	Start: 1850-01-01T00:00:00
	End: 2304-01-01T00:00:00
Meta:
	     climate_model  ensemble_member        model region scenario   unit  \
	0     MAGICCv7.5.1                0  unspecified  World   ssp370  ZJ/yr   
	1     MAGICCv7.5.1                1  unspecified  World   ssp370  ZJ/yr   
	2     MAGICCv7.5.1                2  unspecified  World   ssp370  ZJ/yr   
	3     MAGICCv7.5.1                3  unspecified  World   ssp370  ZJ/yr   
	4     MAGICCv7.5.1                4  unspecified  World   ssp370  ZJ/yr   
	...            ...              ...          ...    ...      ...    ...   
	4795  MAGICCv7.5.1              595  unspecified  World   ssp245  ZJ/yr   
	4796  MAGICCv7.5.1              596  unspecified  World   ssp245  ZJ/yr   
	4797  MAGICCv7.5.1              597  unspecified  World   ssp245  ZJ/yr   
	4798  MAGICCv7.5.1              598  unspecified  World   ssp245  ZJ/yr   
	4799  MAGICCv7.5.1              599  unspeci

#### Convert ocean heat uptake to W/m^2

In [8]:
earth_surface_area = 510.1 * 10 ** 6 * unit_registry("km^2")
magicc_hfds_output = (
    magicc_ohu_output
    .convert_unit("W")
) / earth_surface_area
magicc_hfds_output

<scmdata.ScmRun (timeseries: 4800, timepoints: 455)>
Time:
	Start: 1850-01-01T00:00:00
	End: 2304-01-01T00:00:00
Meta:
	     climate_model  ensemble_member        model region scenario  \
	0     MAGICCv7.5.1                0  unspecified  World   ssp370   
	1     MAGICCv7.5.1                1  unspecified  World   ssp370   
	2     MAGICCv7.5.1                2  unspecified  World   ssp370   
	3     MAGICCv7.5.1                3  unspecified  World   ssp370   
	4     MAGICCv7.5.1                4  unspecified  World   ssp370   
	...            ...              ...          ...    ...      ...   
	4795  MAGICCv7.5.1              595  unspecified  World   ssp245   
	4796  MAGICCv7.5.1              596  unspecified  World   ssp245   
	4797  MAGICCv7.5.1              597  unspecified  World   ssp245   
	4798  MAGICCv7.5.1              598  unspecified  World   ssp245   
	4799  MAGICCv7.5.1              599  unspecified  World   ssp245   
	
	                       unit           variable  
	

### MESMER configuration

In [9]:
# TODO: decide how to join MESMER calibrations with MAGICC output
# @lea you can use whatever methods you want to get params_lt, params_lv, params_gv,
# seeds, land_fractions here, these are just for testing
mesmer_bundle_file = os.path.join(
    "..", "tests", "test-data", "mesmer-bundles", "test-generic-mesmer-bundle.pkl"
)
mesmer_bundle = joblib.load(mesmer_bundle_file)

params_lt = mesmer_bundle["params_lt"]
params_lv = mesmer_bundle["params_lv"]
params_gv_T = mesmer_bundle["params_gv_T"]
seeds = mesmer_bundle["seeds"]
land_fractions = mesmer_bundle["land_fractions"]

## Define run function

Given it's only 150 lines, doing this in the notebook makes it slightly easier to prototype, experiment with parellelisation etc.

In [10]:
def _draw_realisations_from_mesmer_config_and_openscm_output(
    params_lt,
    params_lv,
    params_gv_T,
    seeds,
    land_fractions,
    openscm_gsat,
    openscm_hfds=None,
    n_realisations_per_scenario=5,
):
    """
    Parameters
    ----------
    params_lt : todo
        TODO: description of minimal amount of information required
        
    params_lv : todo
        TODO: description of minimal amount of information required
        
    params_gv_T : todo
        TODO: description of minimal amount of information required
        
    seeds : todo
        TODO: description of minimal amount of information required
        
    land_fractions : todo
        TODO: description of minimal amount of information required
        
    mesmer_bundle_file : str
        File in which the MESMER bundle is saved

    openscm_gsat : :obj:`scmdata.ScmRun`
        Global-mean surface air temperature change output to be used to drive MESMER's emulations

    openscm_hfds : [None, :obj:`scmdata.ScmRun`]
        If supplied, global-mean ocean heat uptake output to be used to drive MESMER's emulations

    n_realisations_per_scenario : int
        Number of realisations to draw for each scenario in ``openscm_gsat`` (and ``openscm_hfds`` if supplied)

    Returns
    -------
    :obj:`xr.Dataset`
        Emulations for each scenario

    Raises
    ------
    ValueError
        The scenarios in `openscm_gsat`` and ``openscm_hfds`` (if supplied) aren't the same

    ValueError
        [TODO talk to Lea about what to do here] The scenarios in ``openscm_gsat`` don't match those for which MESMER has been calibrated

    ValueError
        [TODO talk to Lea about what to do here] The scenarios in ``openscm_gsat`` span a longer time than MESMER has been calibrated for

    ValueError
        ``openscm_gsat`` contains variables other than ``"Surface Air Temperature Change"``

    ValueError
        ``openscm_hfds`` contains variables other than ``"Heat Uptake|Ocean"``
    """
    gsat_scenarios = set(openscm_gsat.get_unique_meta("scenario"))
    hfds_scenarios = set(openscm_hfds.get_unique_meta("scenario"))
    if gsat_scenarios != hfds_scenarios:
        raise ValueError(
            "gsat_scenarios: {}, hfds_scenarios: {}".format(
                gsat_scenarios, hfds_scenarios
            )
        )

    openscm_gsat_for_mesmer = _prepare_openscm_gsat(openscm_gsat)
    openscm_hfds_for_mesmer = _prepare_openscm_hfds(openscm_hfds)

    hard_coded_hist_years = range(1850, 2014 + 1)
    hard_coded_scen_years = range(2015, 3000 + 1)

    out = []
    for scen_gsat in openscm_gsat_for_mesmer.groupby("scenario"):
        scenario = scen_gsat.get_unique_meta("scenario", True)
        # if scenario not in mesmer_bundle["time"]:
        #     raise KeyError("No MESMER calibration available for: {}".format(scenario))

        hist_tas = _filter_and_assert_1d(scen_gsat.filter(year=hard_coded_hist_years))
        scen_tas = _filter_and_assert_1d(scen_gsat.filter(year=hard_coded_scen_years))

        openscm_hfds_for_mesmer_scen = openscm_hfds_for_mesmer.filter(scenario=scenario)
        hist_hfds = _filter_and_assert_1d(openscm_hfds_for_mesmer_scen.filter(year=hard_coded_hist_years))
        scen_hfds = _filter_and_assert_1d(openscm_hfds_for_mesmer_scen.filter(year=hard_coded_scen_years))

        # TODO: remove hard-coding and actually map up with what MESMER expects
        preds_lt_scenario = {}
        for predictor in mesmer_bundle["params_lt"]["preds"]:
            if predictor == "gttas":
                hist_vals = hist_tas
                scen_vals = scen_tas
            elif predictor == "gttas2":
                hist_vals = hist_tas ** 2
                scen_vals = scen_tas ** 2
            elif predictor == "gthfds":
                hist_vals = hist_hfds
                scen_vals = scen_hfds

            preds_lt_scenario[predictor] = {"hist": hist_vals, scenario: scen_vals}

        result_scenario = mesmer.create_emulations.make_realisations(
            preds_lt=preds_lt_scenario,
            params_lt=params_lt,
            params_lv=params_lv,
            params_gv_T=params_gv_T,
            n_realisations=n_realisations_per_scenario,
            seeds=seeds,
            land_fractions=land_fractions,
            time={
                "hist": scen_gsat.filter(year=hard_coded_hist_years)["year"].values,
                scenario: scen_gsat.filter(year=hard_coded_scen_years)["year"].values
            },
        )

        result_scenario = (
            result_scenario
            .squeeze(dim="scenario", drop=True)
            .expand_dims({"scenario": [scenario]})
        )
        out.append(result_scenario)

    out = xr.merge(out)

    return out


def _prepare_openscm_gsat(openscm_gsat):
    return openscm_gsat.convert_unit("K")


def _prepare_openscm_hfds(openscm_hfds):
    return openscm_hfds.convert_unit("W/m^2")


def _filter_and_assert_1d(scmrun, **kwargs):
    filtered = scmrun.filter(**kwargs).values.squeeze()
    if len(filtered.shape) != 1:
        raise ValueError(
            "Filters gave non-1D data: {}, {}".format(filtered.shape, kwargs)
        )

    return filtered


In [18]:
# TODO: add ability to control MESMER print statements
magicc_output = scmdata.run_append([magicc_gsat_output, magicc_hfds_output]).filter(year=YEARS_TO_EMULATE)

all_output = []
for i, em in enumerate(magicc_output.groupby("ensemble_member")):
    result_em = _draw_realisations_from_mesmer_config_and_openscm_output(
        params_lt,
        params_lv,
        params_gv_T,
        seeds,
        land_fractions,
        openscm_gsat=em.filter(variable="Surface Air Temperature Change", year=YEARS_TO_EMULATE),
        openscm_hfds=em.filter(variable="Heat Uptake|Ocean", year=YEARS_TO_EMULATE),
        n_realisations_per_scenario=N_REALISATIONS_PER_SCENARIO,
    ).expand_dims(magicc_ensemble_member=em.get_unique_meta("ensemble_member"))
    
    all_output.append(result_em)
    
    if i % 50 == 1:
        print(i)
#     if i > 4:
#         break

Start with OLS
Start with AR(1) with spatially correlated innovations.
Draw the innovations
Compute the contribution to emus_lv by the AR(1) process with the spatially correlated innovations
Create the full local variability emulations
Start with OLS
Start with AR(1) with spatially correlated innovations.
Draw the innovations
Compute the contribution to emus_lv by the AR(1) process with the spatially correlated innovations
Create the full local variability emulations
Start with OLS
Start with AR(1) with spatially correlated innovations.
Draw the innovations
Compute the contribution to emus_lv by the AR(1) process with the spatially correlated innovations
Create the full local variability emulations
Start with OLS
Start with AR(1) with spatially correlated innovations.
Draw the innovations
Compute the contribution to emus_lv by the AR(1) process with the spatially correlated innovations
Create the full local variability emulations
Start with OLS
Start with AR(1) with spatially correlate

Start with OLS
Start with AR(1) with spatially correlated innovations.
Draw the innovations
Compute the contribution to emus_lv by the AR(1) process with the spatially correlated innovations
Create the full local variability emulations
Start with OLS
Start with AR(1) with spatially correlated innovations.
Draw the innovations
Compute the contribution to emus_lv by the AR(1) process with the spatially correlated innovations
Create the full local variability emulations
Start with OLS
Start with AR(1) with spatially correlated innovations.
Draw the innovations
Compute the contribution to emus_lv by the AR(1) process with the spatially correlated innovations
Create the full local variability emulations
Start with OLS
Start with AR(1) with spatially correlated innovations.
Draw the innovations
Compute the contribution to emus_lv by the AR(1) process with the spatially correlated innovations
Create the full local variability emulations
Start with OLS
Start with AR(1) with spatially correlate

Start with OLS
Start with AR(1) with spatially correlated innovations.
Draw the innovations
Compute the contribution to emus_lv by the AR(1) process with the spatially correlated innovations
Create the full local variability emulations
Start with OLS
Start with AR(1) with spatially correlated innovations.
Draw the innovations
Compute the contribution to emus_lv by the AR(1) process with the spatially correlated innovations
Create the full local variability emulations
Start with OLS
Start with AR(1) with spatially correlated innovations.
Draw the innovations
Compute the contribution to emus_lv by the AR(1) process with the spatially correlated innovations
Create the full local variability emulations
Start with OLS
Start with AR(1) with spatially correlated innovations.
Draw the innovations
Compute the contribution to emus_lv by the AR(1) process with the spatially correlated innovations
Create the full local variability emulations
Start with OLS
Start with AR(1) with spatially correlate

Start with OLS
Start with AR(1) with spatially correlated innovations.
Draw the innovations
Compute the contribution to emus_lv by the AR(1) process with the spatially correlated innovations
Create the full local variability emulations
Start with OLS
Start with AR(1) with spatially correlated innovations.
Draw the innovations
Compute the contribution to emus_lv by the AR(1) process with the spatially correlated innovations
Create the full local variability emulations
Start with OLS
Start with AR(1) with spatially correlated innovations.
Draw the innovations
Compute the contribution to emus_lv by the AR(1) process with the spatially correlated innovations
Create the full local variability emulations
Start with OLS
Start with AR(1) with spatially correlated innovations.
Draw the innovations
Compute the contribution to emus_lv by the AR(1) process with the spatially correlated innovations
Create the full local variability emulations
Start with OLS
Start with AR(1) with spatially correlate

Start with OLS
Start with AR(1) with spatially correlated innovations.
Draw the innovations
Compute the contribution to emus_lv by the AR(1) process with the spatially correlated innovations
Create the full local variability emulations
Start with OLS
Start with AR(1) with spatially correlated innovations.
Draw the innovations
Compute the contribution to emus_lv by the AR(1) process with the spatially correlated innovations
Create the full local variability emulations
Start with OLS
Start with AR(1) with spatially correlated innovations.
Draw the innovations
Compute the contribution to emus_lv by the AR(1) process with the spatially correlated innovations
Create the full local variability emulations
Start with OLS
Start with AR(1) with spatially correlated innovations.
Draw the innovations
Compute the contribution to emus_lv by the AR(1) process with the spatially correlated innovations
Create the full local variability emulations
Start with OLS
Start with AR(1) with spatially correlate

...

Start with OLS
Start with AR(1) with spatially correlated innovations.
Draw the innovations
Compute the contribution to emus_lv by the AR(1) process with the spatially correlated innovations
Create the full local variability emulations
Start with OLS
Start with AR(1) with spatially correlated innovations.
Draw the innovations
Compute the contribution to emus_lv by the AR(1) process with the spatially correlated innovations
Create the full local variability emulations
Start with OLS
Start with AR(1) with spatially correlated innovations.
Draw the innovations
Compute the contribution to emus_lv by the AR(1) process with the spatially correlated innovations
Create the full local variability emulations
Start with OLS
Start with AR(1) with spatially correlated innovations.
Draw the innovations
Compute the contribution to emus_lv by the AR(1) process with the spatially correlated innovations
Create the full local variability emulations
Start with OLS
Start with AR(1) with spatially correlate

Start with OLS
Start with AR(1) with spatially correlated innovations.
Draw the innovations
Compute the contribution to emus_lv by the AR(1) process with the spatially correlated innovations
Create the full local variability emulations
Start with OLS
Start with AR(1) with spatially correlated innovations.
Draw the innovations
Compute the contribution to emus_lv by the AR(1) process with the spatially correlated innovations
Create the full local variability emulations
Start with OLS
Start with AR(1) with spatially correlated innovations.
Draw the innovations
Compute the contribution to emus_lv by the AR(1) process with the spatially correlated innovations
Create the full local variability emulations
Start with OLS
Start with AR(1) with spatially correlated innovations.
Draw the innovations
Compute the contribution to emus_lv by the AR(1) process with the spatially correlated innovations
Create the full local variability emulations
Start with OLS
Start with AR(1) with spatially correlate

Start with OLS
Start with AR(1) with spatially correlated innovations.
Draw the innovations
Compute the contribution to emus_lv by the AR(1) process with the spatially correlated innovations
Create the full local variability emulations
Start with OLS
Start with AR(1) with spatially correlated innovations.
Draw the innovations
Compute the contribution to emus_lv by the AR(1) process with the spatially correlated innovations
Create the full local variability emulations
Start with OLS
Start with AR(1) with spatially correlated innovations.
Draw the innovations
Compute the contribution to emus_lv by the AR(1) process with the spatially correlated innovations
Create the full local variability emulations
Start with OLS
Start with AR(1) with spatially correlated innovations.
Draw the innovations
Compute the contribution to emus_lv by the AR(1) process with the spatially correlated innovations
Create the full local variability emulations
Start with OLS
Start with AR(1) with spatially correlate

Start with OLS
Start with AR(1) with spatially correlated innovations.
Draw the innovations
Compute the contribution to emus_lv by the AR(1) process with the spatially correlated innovations
Create the full local variability emulations
Start with OLS
Start with AR(1) with spatially correlated innovations.
Draw the innovations
Compute the contribution to emus_lv by the AR(1) process with the spatially correlated innovations
Create the full local variability emulations
Start with OLS
Start with AR(1) with spatially correlated innovations.
Draw the innovations
Compute the contribution to emus_lv by the AR(1) process with the spatially correlated innovations
Create the full local variability emulations
Start with OLS
Start with AR(1) with spatially correlated innovations.
Draw the innovations
Compute the contribution to emus_lv by the AR(1) process with the spatially correlated innovations
Create the full local variability emulations
Start with OLS
Start with AR(1) with spatially correlate

KeyboardInterrupt: 

In [26]:
all_output = xr.combine_by_coords(all_output)
all_output

<xarray.Dataset>
Dimensions:                 (magicc_ensemble_member: 600, realisation: 2, scenario: 8, year: 301, z: 117)
Coordinates:
  * magicc_ensemble_member  (magicc_ensemble_member) int64 0 1 2 ... 597 598 599
  * scenario                (scenario) <U11 'ssp119' 'ssp126' ... 'ssp585'
  * realisation             (realisation) int64 0 1
  * year                    (year) int64 1850 1851 1852 1853 ... 2148 2149 2150
    lat                     (z) float64 -49.5 -40.5 -31.5 ... 76.5 76.5 76.5
    lon                     (z) float64 -72.0 -72.0 -72.0 ... -18.0 90.0 108.0
Dimensions without coordinates: z
Data variables:
    tas                     (magicc_ensemble_member, scenario, realisation, year, z) float64 ...

In [27]:
all_output.to_netcdf(OUT_FILE)
!du -sh {OUT_FILE}

2.6G	probabilistic_example_emulation.nc
